In [1]:
# get ireland weather stations info
# get EFI/CDF and wind gusts EFI for stations
# note it takes a while for the API requests to go through

stations_url = 'https://www.met.ie/climate/weather-observing-stations'

import requests
req = requests.get(stations_url)
if req.ok:
    text = req.text
req.close()


In [2]:
import re
from bs4 import BeautifulSoup
regex = r"""class="collapsed">(.*?)</a>.*?<p><strong>Location</strong><br />(.*?)<br />(.*?)<br />(.*?)m above mean sea level<"""
#regex = r"""class="collapsed">(.*?)</a>.*?<p>"""
res = re.findall(regex, text, flags=re.DOTALL|re.I)

In [3]:
# check res for debugging
# res

In [4]:
def convert_coord_to_decimal(coord):
    [h, m, s, hemisphere] = coord
    matchW = re.search(r'W', hemisphere, flags=re.IGNORECASE)
    matchS = re.search(r'S', hemisphere, flags=re.IGNORECASE)
    # multiply decimal coords by 1 or -1 depending on which hemisphere
    if matchW or matchS:
        positive = -1.0
    else:
        positive = 1.0
    return positive * (
        float(h) + 
        (float(m) / 60.0) +
        (float(s) / 3600.0)
    )

def parse_string_to_coords_hemisphere(str):
    matches = re.findall(r'(\d+)[^\d]+(\d+)[^\d]+(\d+)[^\d]*([NEWS])', str, flags=re.IGNORECASE)
    return matches[0]

def convert_str_to_decimal(str):
    return convert_coord_to_decimal(parse_string_to_coords_hemisphere(str))

print("Location, Lat, Long, Altitude (m)\n")
ireland_weather_locations = {}
for [location, lat, long, alt] in res:
    location = BeautifulSoup(location).get_text().strip()
    print(location)
    lat = BeautifulSoup(lat).get_text().strip()
    long = BeautifulSoup(long).get_text().strip()
    lat_dec = convert_str_to_decimal(lat)
    long_dec = convert_str_to_decimal(long)
    print(f"{lat_dec:.4f}")
    print(f"{long_dec:.4f}")
    alt = BeautifulSoup(alt).get_text().strip()
    print(alt)
    print("")
    ireland_weather_locations[location] = [lat_dec, long_dec, alt]
    

Location, Lat, Long, Altitude (m)

Athenry
53.2892
-8.7856
40

Ballyhaise
54.0514
-7.3097
78

Belmullet
54.2275
-10.0069
9

Carlow Oakpark
52.8611
-6.9153
62

Claremorris
53.7108
-8.9925
68

Dunsany
53.5158
-6.6600
83

Fermoy Moorepark
52.1639
-8.2639
46

Finner
54.4939
-8.2431
33

Gurteen
53.0531
-8.0086
75

Johnstown Castle
52.2978
-6.4967
62

Mace Head
53.3258
-9.9008
21

Malin Head
55.3722
-7.3389
22

Markree
54.1750
-8.4556
34

Mount Dillon
53.7269
-7.9808
39

Mullingar
53.5372
-7.3622
101

Newport
53.9222
-9.5722
22

Phoenix Park
53.3639
-6.3333
48

Roches Point
51.7931
-8.2444
43

Sherkin Island
51.4764
-9.4278
21

Valentia
51.9397
-10.2444
25



In [5]:

api_url = "https://charts.ecmwf.int/opencharts-api/v1/"
print('{}products/'.format(api_url))
import requests

result =requests.get('{}products/'.format(api_url))

all_data = result.json()

https://charts.ecmwf.int/opencharts-api/v1/products/


In [6]:
import json
d = json.dumps(all_data, indent=4)
#print(d)

In [7]:
# wind gusts
openchart_efi_gusts = 'efi2web_10fg'
schema_url_openchart_efi_gusts = [x['schema-url'] for x in all_data if x['name'] == openchart_efi_gusts][0]
product_url_openchart_efi_gusts = [x['product-url'] for x in all_data if x['name'] == openchart_efi_gusts][0]
result = requests.get(schema_url_openchart_efi_gusts)
schema_openchart_efi_gusts = result.json()
d = json.dumps(schema_openchart_efi_gusts, indent=4)
print(d)

{
    "openapi": "3.0.2",
    "info": {
        "version": "1.0",
        "title": "Export Opencharts"
    },
    "servers": [
        {
            "url": "https://charts.ecmwf.int/opencharts-api/v1/"
        }
    ],
    "paths": {
        "/products": {
            "get": {
                "responses": {
                    "200": {
                        "content": {
                            "application/json": {
                                "schema": {
                                    "$ref": "#/components/schemas/products-list"
                                }
                            }
                        },
                        "description": "Data are ready to be downloaded"
                    }
                }
            }
        },
        "/products/efi2web_10fg/": {
            "get": {
                "responses": {
                    "default": {
                        "content": {
                            "application/json": {
            

In [8]:
# efi/cdf
openchart_efi_cdf = 'opencharts_efi-cdf-meteogram'
schema_url_openchart_efi_cdf = [x['schema-url'] for x in all_data if x['name'] == openchart_efi_cdf][0]
product_url_openchart_efi_cdf = [x['product-url'] for x in all_data if x['name'] == openchart_efi_cdf][0]
result = requests.get(schema_url_openchart_efi_cdf)
schema_openchart_efi_cdf = result.json()
d = json.dumps(schema_openchart_efi_cdf, indent=4)
#print(d)

In [9]:
print("Parameters for EFI GUSTS")
parameters_efi_gusts = schema_openchart_efi_gusts['paths']['/products/' + openchart_efi_gusts + '/']['get']['parameters']
parameter_names_efi_gusts = [x['name'] for x in parameters_efi_gusts]
print(parameter_names_efi_gusts)
print("")
print("Parameter schema:")
#print(json.dumps(parameters_efi_gusts,indent=4))

Parameters for EFI GUSTS
['format', 'base_time', 'quantile', 'day', 'area']

Parameter schema:


In [10]:
print("Parameters for EFI CDF")
parameters_efi_cdf = schema_openchart_efi_cdf['paths']['/products/' + openchart_efi_cdf + '/']['get']['parameters']
parameter_efi_cdf_names = [x['name'] for x in parameters_efi_cdf]
print(parameter_efi_cdf_names)
print("")
print("Parameter schema:")
#print(json.dumps(parameters_efi_cdf,indent=4))

Parameters for EFI CDF
['format', 'base_time', 'step', 'valid_time', 'city', 'altitude', 'lon', 'lat']

Parameter schema:


In [11]:
print("API urls")
print(product_url_openchart_efi_cdf)
print(product_url_openchart_efi_gusts)

API urls
https://charts.ecmwf.int/opencharts-api/v1/products/opencharts_efi-cdf-meteogram/
https://charts.ecmwf.int/opencharts-api/v1/products/efi2web_10fg/


In [12]:
from IPython.display import display, HTML

def get_EFI_gusts(day, area):
    params = {'day': day, 'area': area}
    result = requests.get(product_url_openchart_efi_gusts, params=params)
    if result.status_code == 200:
        data = result.json()
        url = data['data']['link']['href']
        return url
    else:
        return None

def get_EFI_CDF(valid_time, lat, lon, altitude=None):
    if altitude is None:
        params = {'valid_time': valid_time, 'lat': lat, 'lon': lon}
    else:
        params = {'valid_time': valid_time, 'lat': lat, 'lon': lon, 'altitude': altitude}
    result = requests.get(product_url_openchart_efi_cdf, params=params)
    if result.status_code == 200:
        data = result.json()
        url = data['data']['link']['href']
        return url
    else:
        return None

In [13]:
days = list(range(1,8))
# EFI gusts only has days 1-7
#days.append("7-9")
#days.append("10-15")
area = "North West Europe"

from IPython.display import HTML, display

# for sequentially displaying
"""
print("ECMWF EFI gusts for NW Europe\n")
html_code = ""
for day in days:
    url = get_EFI_gusts(day, area)
    html_code += f"<p>ECMWF EFI gusts for day {day} \n</p><img src='{url}'/>"

display(HTML(html_code))
"""

image_paths_gusts = []
for day in days:
    url = get_EFI_gusts(day, area)
    if url:
        image_paths_gusts.append(url)
    else:
        print(f"Error retrieving data for day {day}")

In [14]:
# Create HTML code for the image slider and buttons
html_code = """
<div id="imageGusts-controls">
    <button onclick="previousGustsImage()">Previous</button>
    <input type="range" id="imageGusts-slider" min="0" value="0" onchange="updateGustsImage()">
    <button onclick="nextGustsImage()">Next</button>
</div>

<div id="imageGusts-container">
    <img src="" id="displayedGusts-image" style="width: 100%;">
</div>

<script>
    var currentImageGustsIndex = 0;
    var imageGustsPaths = """ + str(image_paths_gusts) + """;
    var displayedGustsImage = document.getElementById("displayedGusts-image");
    var imageGustsSlider = document.getElementById("imageGusts-slider");

    function displayImageGusts(index) {
        var imageGusts_url = imageGustsPaths[index];
        displayedGustsImage.src = imageGusts_url;
        currentImageGustsIndex = index;
        imageGustsSlider.max = imageGustsPaths.length - 1;
        imageGustsSlider.value = index;
    }

    function previousGustsImage() {
        currentImageGustsIndex = (currentImageGustsIndex - 1 + imageGustsPaths.length) % imageGustsPaths.length;
        displayImage(currentImageGustsIndex);
    }

    function nextGustsImage() {
        currentImageGustsIndex = (currentImageGustsIndex + 1) % imageGustsPaths.length;
        displayImageGusts(currentImageGustsIndex);
    }

    function updateGustsImage() {
        currentImageGustsIndex = parseInt(imageGustsSlider.value);
        displayImageGusts(currentImageGustsIndex);
    }

    // Initial display
    displayImageGusts(currentImageGustsIndex);
</script>
"""

# Display the initial image and slider
display(HTML(html_code))

In [39]:
valid_time = "2023-09-28T00:00:00Z"


from IPython.display import HTML, display
# for sequentially displaying
"""
print("ECMWF EFI/CDF for Ireland Weather Stations @ https://www.met.ie")
print(f"For end time: {valid_time}\n")
html_code = ""
for station in ireland_weather_locations:
    [lat, lon, altitude] = ireland_weather_locations[station]
    url = get_EFI_CDF(valid_time, lat, lon, altitude)
    html_code += f"<p>ECMWF EFI for {station} \n</p><img src='{url}'/>"

display(HTML(html_code))
"""

image_paths_cdf = []
image_captions_cdf = []
for station in ireland_weather_locations:
    [lat, lon, altitude] = ireland_weather_locations[station]
    url = get_EFI_CDF(valid_time, lat, lon, altitude)
    if url:
        image_paths_cdf.append(url)
        image_captions_cdf.append(f"ECMWF EFI for {station}\nFor end time: {valid_time}")
    else:
        print(f"Error retrieving data for station: {station}")

In [40]:
# Create HTML code for the image slider and buttons
html_code_cdf = """
<div id="CDF-mage-controls">
    <button onclick="CDFpreviousImage()">Previous</button>
    <input type="range" id="CDF-image-slider" min="0" value="0" onchange="CDFupdateImage()">
    <button onclick="CDFnextImage()">Next</button>
</div>

<div id="CDF-image-caption">
    <p>Caption for Image 1</p>
</div>

<div id="CDF-image-container">
    <img src="" id="CDF-displayed-image" style="width: 100%;">
</div>

<script>
    var currentImageIndexCDF = 0;
    var imagePathsCDF = """ + str(image_paths_cdf) + """;
    var imageCaptionsCDF = """ + str(image_captions_cdf) + """;
    var displayedImageCDF = document.getElementById("CDF-displayed-image");
    var imageSliderCDF = document.getElementById("CDF-image-slider");
    var imageCaptionCDF = document.getElementById("CDF-image-caption");

    function CDFdisplayImageAndCaption(index) {
        console.log(imagePathsCDF)
        var image_url = imagePathsCDF[index];
        var caption = imageCaptionsCDF[index];
        displayedImageCDF.src = image_url;
        imageCaptionCDF.innerHTML = "<p>" + caption + "</p>";
        currentImageIndexCDF = index;
        imageSliderCDF.max = imagePathsCDF.length - 1;
        imageSliderCDF.value = index;
    }

    function CDFpreviousImage() {
        currentImageIndexCDF = (currentImageIndexCDF - 1 + imagePathsCDF.length) % imagePathsCDF.length;
        CDFdisplayImageAndCaption(currentImageIndexCDF);
    }

    function CDFnextImage() {
        currentImageIndexCDF = (currentImageIndexCDF + 1) % imagePathsCDF.length;
        CDFdisplayImageAndCaption(currentImageIndexCDF);
    }

    function CDFupdateImage() {
        currentImageIndexCDF = parseInt(imageSliderCDF.value);
        CDFdisplayImageAndCaption(currentImageIndexCDF);
    }

    // Initial display
    CDFdisplayImageAndCaption(currentImageIndexCDF);
</script>
"""

# Display the initial image, caption, and slider
display(HTML(html_code_cdf))